In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_privacy
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import cifar10
from tensorflow_privacy.privacy.analysis import compute_dp_sgd_privacy
%matplotlib inline

In [2]:
# Loading cifar10 image dataset
data_train, data_test = cifar10.load_data()
x_train, y_train = data_train
x_test, y_test = data_test

# Normalizing pixel values of images
x_train = x_train / 255.0
x_test = x_test / 255.0

# Flattening images in dataset
y_train = y_train.flatten().reshape(y_train.shape[0], 1)
y_test = y_test.flatten().reshape(y_test.shape[0], 1)

# One hot encoding of labels/target column
# y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
# y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)

print("training:", x_train.shape, y_train.shape)
print("testing:", x_test.shape, y_test.shape)

training: (50000, 32, 32, 3) (50000, 1)
testing: (10000, 32, 32, 3) (10000, 1)


In [3]:
## Model1: Simple CNN

In [4]:
## Model2: ResNet with transfer learning

In [5]:
## Model3: HOG classification